# Lemmatisierung mit regulären Ausdrücken ... zumindest fast
In dieser Übung geht es darum, die Flexionsaffixe von Verben zu entfernen. Dies ist ein erster Schritt zur Lemmatisierung, also dazu, konkrete Wortformen auf ihre Zitierformen im Lexikon zurückzuführen. Wir werden dabei sehen, dass es mit diesem Schritt oft nicht getan ist, dass ordentliche Lemmatisierung also noch ein gutes Stück schwieriger ist.

## Nötige Pakete importieren
Um eine TSV-Datei zu öffnen, verwenden wir hier *pandas*, weil wir damit leichter auf einzelne Tabellenspalten zuzugreifen. Achtung, Pandas ist nicht teil der Standardbibliothek und muss separat installiert werden. Das sollte in den meisten Python3-Umgebungen über den Python-eigenen Paketmanager `pip3` möglich sein. Um Pandas zu installieren muss nur der Befehl `pip3 install --user pandas` in einem Terminal ausgeführt werden.

Sollte die Installation von Pandas nicht möglich sein, kann die TSV-Datei auch über das `csv`-Modul der Standardbibliothek eingelesen werden. Eine Anleitung, wie das möglich ist, gibt es z. B. [hier](https://medium.com/@adds68/parsing-tsv-file-with-csv-in-python-662d6347b0cd).

Außerdem brauchen wir natürlich reguläre Ausdrücke.

In [2]:
import pandas as pd
import re

## Tabelle einlesen
Die Datei "uebung_verben.tsv" enthält 500 Treffer aus einer Suche nach Vollverben im DWDS-Kernkorpus. (Die Abfrage sieht so aus: <https://www.dwds.de/r?q=%24p%3DVV*+%23sep&corpus=kern&date-start=1900&date-end=1999&genre=Belletristik&genre=Wissenschaft&genre=Gebrauchsliteratur&genre=Zeitung&format=full&sort=random&limit=10>)

Neben den konkreten Wortformen, die von der Abfrage gefunden wurden, enthält die Datei auch für jede Form den kompletten Satz, in dem sie vorkommt, sowie Informationen zur Textquelle. Jede Information befindet sich in einer eigenen Spalte der Tabelle.

Da Spalten in dieser Tabelle mit Tabstops (\t) getrennt sind, bietet sich read_table() an.

In [4]:
path = "uebung_verben.tsv"
table = pd.read_table(path)
table

,match,sentence,collection,author,title,textClass,bibl,date
0,entsteht,"Es bleibt wohl dem , der die Frage zu stellen ...",kern,"Sloterdijk, Peter",Kritik der zynischen Vernunft,Wissenschaft,"Sloterdijk, Peter: Kritik der zynischen Vernun...",1983-12-31
1,verloren,"Diese Sumpfreminiszenzen , die mich nicht herg...",kern,"May, Karl",Mein Leben und Streben,Gebrauchsliteratur::Autobiographien,"May, Karl: Mein Leben und Streben, Selbstbiogr...",1910-12-31
2,abschaffen,Hitler ist schon seit langen Monaten deutscher...,kern,Kurt Tucholsky:Kaspar Hauser,Hitler und Goethe,Zeitung,"Tucholsky, Kurt: Hitler und Goethe. In: Kurt T...",1932-05-17
3,hielt,"Warum tat er es nicht , sondern hielt an sich ...",kern,"Huch, Ricarda",Der große Krieg in Deutschland,Belletristik::Gebrauchsliteratur,"Huch, Ricarda: Der Dreißigjährige Krieg, Wiesb...",1914-12-31
4,bewahren,"Er besitzt eigentlich alles , was eine Frau vo...",kern,"Strauß, Botho",Der junge Mann,Belletristik::Roman,"Strauß, Botho: Der junge Mann, München: Hanser...",1984-12-31
5,gesagt,"Ein Idiot hat mir gesagt , warum ich mich zu H...",kern,"Döblin, Alfred",Berlin Alexanderplatz(Die Geschichte vom Franz...,Belletristik::Roman,"Döblin, Alfred: Berlin Alexanderplatz, Olten: ...",1929-12-31
6,verkündet,Es zeigte ein von Stacheldraht umzäuntes Stück...,kern,unknown,Scharfe Auseinandersetzungen KPF/KPdSU wegen F...,Zeitung::FRANKREICH. SOWJETUNION. KOMMUNISMUS.,"Archiv der Gegenwart, 2001 [1976]",1976-01-03
7,krallte,"Er heulte laut auf , einmal um das Pferd , dan...",kern,"Welk, Ehm",Die Heiden von Kummerow,Belletristik,"Welk, Ehm: Die Heiden von Kummerow, Rostock: H...",1937-12-31
8,wohnt,"In den letzten Jahren , zum Beispiel , wohnt e...",kern,"Tucholsky, Kurt:Kaspar Hauser",Der Andere,Zeitung,"Tucholsky, Kurt: Der Andere. In: ders., Kurt T...",1927-11-29
9,eingetroffen,- Der Luftschiffer Graf Zeppelin ist aus Stutt...,kern,unknown,NaN,Zeitung::Abend-Ausgabe,"Vossische Zeitung (Abend-Ausgabe), 04.03.1908",1908-03-04


## Verbformen

Wir brauchen hier eigentlich nur die erste Spalte ...

In [7]:
verben = table["match"]
verben

0           entsteht
1           verloren
2         abschaffen
3              hielt
4           bewahren
5             gesagt
6          verkündet
7            krallte
8              wohnt
9       eingetroffen
10             zieht
11            meldet
12             hören
13             erhob
14             griff
15            stoben
16          träumend
17          geraucht
18           schwieg
19           notiert
20        geschminkt
21          betrifft
22           gemacht
23            führen
24          gekriegt
25       übergesetzt
26              rief
27           ablegte
28            stimme
29            kommen
           ...      
470           sengen
471        beschwert
472            legte
473         erworben
474         erlauben
475         gedrillt
476           sangen
477           merkte
478    vorausgesetzt
479          lauerte
480          erteilt
481          anzutun
482          drangen
483       gestaltete
484          bringen
485         knickten
486          

Das ist nun strenggenommen ein spezieller Objekttyp aus *pandas* (nämlich *Series*), aber wir können ganz normal damit arbeiten.

Die Aufgabe besteht nun darin, den untenstehenden Code so anzupassen, dass bei jeder Verbform Flexionsaffixe entfernt werden.  Eine kurze Übersicht über Verbkonjugation im Deutschen gibt es z.B. bei Wikipedia: <https://de.wikipedia.org/wiki/Konjugation_(Grammatik)>

Finde also einen passenden regulären Ausdruck!

Welche Probleme treten dabei auf? Welche weiteren Schritte müssten für eine vollständige Lemmatisierung noch umgesetzt werden?

In [8]:
for verb in verben:
    print(verb)

entsteht
verloren
abschaffen
hielt
bewahren
gesagt
verkündet
krallte
wohnt
eingetroffen
zieht
meldet
hören
erhob
griff
stoben
träumend
geraucht
schwieg
notiert
geschminkt
betrifft
gemacht
führen
gekriegt
übergesetzt
rief
ablegte
stimme
kommen
anvertraut
heiraten
begann
verfolgen
konferierte
kostete
gesehen
erkennen
setzen
gespuckt
polierten
erregt
darstellt
versuchen
geißelte
gegründet
gab
kommt
voraussehen
unterzeichnen
gehangen
aufschrie
angewachsen
erwiderte
verringern
verlangen
paßte
zugelassen
erklären
verzichten
reduziert
gibt
ausgezogen
merken
gerührt
gellt
behalten
verheißt
wende
weiß
wandeln
bekundet
enthält
einbringe
fordern
angeredet
lassen
genommen
siedelte
sagen
verschwand
gerichtet
geführt
trennen
nahm
drehen
wähle
besucht
abgeändert
überreicht
geben
zusammengenommen
aufwies
arbeitet
erörtern
macht
ging
folgt
sträubte
zeigt
unterliegen
bewegte
wirft
formuliert
findet
ankommt
zählt
sehen
gestopft
bearbeitet
auswirken
abschnitten
vollud
begegneten
garantieren
fällt
verkünde